In [1]:
import torch
import torch.nn as nn # All neural nets 
import torch.optim as optim # Optimization algos
import torch.nn.functional as F # All non parametric funcs - relu etc. better to use from here than nn
from torch.utils.data import DataLoader # Dataset management
import torchvision.datasets as datasets # MNIST is here
import torchvision.transforms as transforms # Has transforms that we need
from tqdm import tqdm

In [2]:
# Fully connected network
# input = 784 (28*28)
class NN(nn.Module):
    def __init__(self, input_size, num_classes): 
        super(NN, self).__init__()
        self.fc1 = nn.Linear(input_size, 50)
        self.fc2 = nn.Linear(50, num_classes)
    
    def forward(self, x):
        x = F.relu(self.fc1(x))
        x = self.fc2(x)
        return x

In [3]:
# rand test
model = NN(784, 10)
x = torch.randn(64, 784) # 64 images - minibatch
print(model(x).shape)

torch.Size([64, 10])


In [4]:
# set device
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [5]:
# Hyperparams
input_size = 784
num_classes = 10
learning_rate = 0.001
batch_size = 64
num_epochs = 5

In [6]:
# Data
# transform is to convert from numpy (by default) to tensors
train_dataset = datasets.MNIST(root='dataset/', train=True, transform=transforms.ToTensor(), download=True)
train_loader = DataLoader(dataset=train_dataset, batch_size=batch_size, shuffle=True)

test_dataset = datasets.MNIST(root='dataset/', train=False, transform=transforms.ToTensor(), download=True)
test_loader = DataLoader(dataset=test_dataset, batch_size=batch_size, shuffle=True)

In [7]:
# Init model
model = NN(input_size=input_size, num_classes=num_classes).to(device)

In [8]:
# Loss and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=learning_rate)

In [9]:
# Train
for epoch in tqdm(range(num_epochs), desc="Training Progress"):
    for batch_idx, (data, targets) in enumerate(train_loader):
        data = data.to(device=device)
        targets = targets.to(device=device)
        
        data = data.reshape(data.shape[0], -1) # Changes from (64,1,28,28) to (64,784)
        
        scores = model(data)
        loss = criterion(scores, targets)

        optimizer.zero_grad() # Do so that prev loops forwarrd loss isnt used
        loss.backward()

        # gradient descent or adam step
        optimizer.step()

Training Progress: 100%|██████████| 5/5 [00:29<00:00,  5.82s/it]


In [10]:
# accuracy

def check_accuracy(loader, model):
    if loader.dataset.train:
        print("Train Dataset\n")
    else:
        print("Test Dataset\n")
    num_correct = 0
    num_samples = 0
    model.eval()

    with torch.no_grad(): # no need to compute gradient
        for x, y in loader:
            x = x.to(device=device)
            y = y.to(device=device)

            x = x.reshape(x.shape[0], -1)

            scores = model(x)
            _, predictions = scores.max(1)

            num_correct += (predictions == y).sum()

            num_samples += predictions.size(0)

    # model.train()
    return num_correct / num_samples

In [12]:
print(float(check_accuracy(train_loader, model)))
print("\n")
print(float(check_accuracy(test_loader, model)))

Train Dataset

0.973466694355011


Test Dataset

0.9666999578475952
